<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
#библиотеки
import pandas as pd
import numpy as np

#модели
from sklearn.linear_model import LinearRegression

#метрики
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
features = data.drop('Страховые выплаты',axis=1)
features.insert(0,'w0',pd.Series([1]*features.shape[0]))
target = data['Страховые выплаты']

<b>Вывод по загрузке данных</b></p>
Представлены данные по 5000 клиентов.</p> 

Пропуски, выбросы и заведомо некорректные значения отсутствуют.</p>

Предобработка данных не требуется

## Умножение матриц

Рассмотрим возможность умножения исходной матрицы на обратимую без потери качества работы модели регрессии:

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** 
При умножении матрицы признаков на обратимую матрицу результат отработки модели не изменится:
$$
a`=a
$$

**Обоснование:** 
умножим матрицу признаков z на корректирующую матрицу p
$$
xp = z
$$
$$
w`=(z^Tz)^{-1} z^T y
$$
$$
(z^Tz)^{-1} z^T y = ((xp)^{T}(xp))^{-1} (xp)^{T} y = (p^{T}x^{T}xp)^{-1} p^{T}x^{T} y = (x^{T}x)^{-1}(p^{T}p)^{-1} p^{T}x^{T} y = p^{-1} (x^{T}x)^{-1}(p^{T})^{-1}p^{T}x^{T} y = p^{-1}(x^{T}x)^{-1}x^{T} y = p^{-1}w
$$
$$
a=xw
$$
$$
a`=zw`= xp p^{-1}w=xEw=a
$$
**Вывод**
$$
a`=a
$$
По результату умножения матрицы признаков на обратимую матрицу результат отработки модели не изменяется

**Алгоритм**


в исходной матрице "признаков" 5 столбца. Воспользуемся для умножения обратимой матрицей размерностью 5х5:
1. Создадим модель на базе библиотеки sklern.LinerRegression()
2. Создаем обратимую матрицу средствами Numpy
3. Проверяем оборатимость матрицы
4. Тестируем отработку обученной модели как на исходной матрице признаков, так и на матрице признаков умноженной на созданную обратимую матрицу.
5. Сравниваем результаты оценки полученной средствами r2_score

**Обоснование**

в пункте 2 приведено доказательство возможности домножения матрицы признаков без потери эффективности использования модели. Таким образом качество предсказания по матрице признаков, умноженной на обратимую будет равно качеству предсказания по исходной матрице.

## Проверка алгоритма

In [6]:
#создаем обратимую матрицу
inverse = []
P = np.random.normal(size=(features.shape[1], features.shape[1]))
#проверяем на обратимость
try:
    inverse = np.linalg.inv(P)
    print('Reversibility Checked successfully')
except np.linalg.LinAlgError:
    # Not invertible. Skip this one.
    pass
print('Show the E matrix')
#print(np.dot(P,inverse).astype('int64'))
print((P@inverse).astype('int64'))

Reversibility Checked successfully
Show the E matrix
[[0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]


In [7]:
model = LinearRegression()

In [8]:
model.fit(features,target)

LinearRegression()

In [9]:
predictions = model.predict(features)
print('R2 для первичной матрицы {}'.format(r2_score(target, predictions)))

R2 для первичной матрицы 0.42494550286668


In [10]:
features_new = features@P
model.fit(features_new,target)
predictions1 = model.predict(features_new)
print('R2 для матрицы умноженной на обратимую {}'.format(r2_score(target, predictions1)))

R2 для матрицы умноженной на обратимую 0.4249453781096372


<b>Вывод </b> мы осуществили изменение/сделали нечитаемыми исходные данные о клиентах сохранив пригодность их использования для обучения модели линейной регрессии и использования ее для предсказания целевого признака. Метрика R2 сохранилась на прежнем уровне.